<a href="https://colab.research.google.com/github/theAkhileshRai/DeepLearning/blob/master/AssetLvl_Pred_lNN_BinaryClassifierDCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder    


from sklearn.metrics import confusion_matrix, classification_report

from keras.utils import to_categorical

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

In [63]:
df = pd.read_csv('/content/data_full_asset.csv',index_col=False)

In [64]:
df.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,multi_class,Binary,stage,Unnamed: 0
0,01-12-2019 00:30,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238676,820.900548,1357.93250,Ashland_Unit256,Normal,0.0,3.0,NaN
1,01-12-2019 01:30,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,Normal,0.0,3.0,NaN
2,01-12-2019 02:30,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,Normal,0.0,3.0,NaN
3,01-12-2019 03:30,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,Normal,0.0,3.0,NaN
4,01-12-2019 04:30,1.0,109.567843,227.339240,351.392952,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,Normal,0.0,3.0,NaN


In [65]:
df['s_temp_mean*s_pressure_mean'] = df['s_pressure_mean']*df['s_temp_mean']
df['d_temp_mean*d_pressure_mean'] = df['d_pressure_mean']*df['d_temp_mean']

In [66]:
df['s_temp_crank*s_pressure_crank'] = df['s_temp_crank']*df['s_pressure_crank']
df['d_temp_crank*d_pressure_crank'] = df['d_pressure_crank']*df['d_temp_crank']

In [67]:
df.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,multi_class,Binary,stage,Unnamed: 0,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,s_temp_crank*s_pressure_crank,d_temp_crank*d_pressure_crank
0,01-12-2019 00:30,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238676,820.900548,1357.93250,Ashland_Unit256,Normal,0.0,3.0,NaN,35495.693668,178966.511738,36568.496401,181230.701112
1,01-12-2019 01:30,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,Normal,0.0,3.0,NaN,34287.262879,176957.908138,35230.156162,179385.590107
2,01-12-2019 02:30,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,Normal,0.0,3.0,NaN,33953.168175,177511.639298,34687.769610,179829.918423
3,01-12-2019 03:30,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,Normal,0.0,3.0,NaN,36123.580608,185232.534888,36900.189126,187543.369569
4,01-12-2019 04:30,1.0,109.567843,227.339240,351.392952,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,Normal,0.0,3.0,NaN,37697.983717,191259.228667,38501.367781,193516.619913


In [68]:
# df1 = pd.read_csv('/content/old_normalised_assets-2.csv',index_col=False)
# df2 = pd.read_csv('/content/new_normalised_assets-2.csv',index_col=False)

In [69]:
df['Asset'].unique()

array(['Ashland_Unit256', 'Center station_C2', 'El Dorado_Unit J2502',
       'Saltlake_K71010B', 'West_Turkey_Unit2', 'Cross bar_Unit1',
       'Reagan_1_Unit6', 'Reagan_1_Unit3', 'Lightning_Unit2',
       'Hobit_Unit2', 'Katy_Unit1116', 'Katy_Unit1117', 'HobitLP_Unit102',
       'Goliad_Unit1100', 'Lightning_Unit4', 'Ashland_256',
       'Clark county_6', 'RJ booster_1', 'Strawberry_3.csv'], dtype=object)

In [70]:
# 'Clark county_6', 'RJ booster_1', 'Strawberry_3.csv'

In [71]:
df2 = df.loc[(df.Asset == 'Clark county_6') | (df.Asset == 'RJ booster_1') | (df.Asset == 'Strawberry_3')|(df.Asset == 'Lightning_Unit2')]

In [72]:
df2.shape

(5223, 21)

In [73]:
df2["Binary"].unique()

array([0., 1.])

In [74]:
#  
#        
#        

In [75]:
df1 = df.loc[(df.Asset == 'West_Turkey_Unit2') | (df.Asset == 'Cross bar_Unit1') | (df.Asset == 'Reagan_1_Unit6') | (df.Asset == 'Reagan_1_Unit3')|
        (df.Asset == 'Hobit_Unit2')|(df.Asset == 'Katy_Unit1116')|(df.Asset == 'Katy_Unit1117')|
       (df.Asset == 'HobitLP_Unit102')|(df.Asset == 'Goliad_Unit1100')|(df.Asset == 'Lightning_Unit4')|(df.Asset == 'Ashland_Unit256') | 
       (df.Asset == 'Center station_C2') | (df.Asset == 'El Dorado_Unit J2502') | (df.Asset == 'Saltlake_K71010B')]

In [76]:
df1.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,multi_class,Binary,stage,Unnamed: 0,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,s_temp_crank*s_pressure_crank,d_temp_crank*d_pressure_crank
0,01-12-2019 00:30,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238676,820.900548,1357.93250,Ashland_Unit256,Normal,0.0,3.0,NaN,35495.693668,178966.511738,36568.496401,181230.701112
1,01-12-2019 01:30,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,Normal,0.0,3.0,NaN,34287.262879,176957.908138,35230.156162,179385.590107
2,01-12-2019 02:30,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,Normal,0.0,3.0,NaN,33953.168175,177511.639298,34687.769610,179829.918423
3,01-12-2019 03:30,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,Normal,0.0,3.0,NaN,36123.580608,185232.534888,36900.189126,187543.369569
4,01-12-2019 04:30,1.0,109.567843,227.339240,351.392952,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,Normal,0.0,3.0,NaN,37697.983717,191259.228667,38501.367781,193516.619913


In [77]:
df1.columns

Index(['time', 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean', 'Asset',
       'multi_class', 'Binary', 'stage', 'Unnamed: 0',
       's_temp_mean*s_pressure_mean', 'd_temp_mean*d_pressure_mean',
       's_temp_crank*s_pressure_crank', 'd_temp_crank*d_pressure_crank'],
      dtype='object')

In [78]:
df1['Binary'] = df1['Binary'].fillna(0)
df1['multi_class'] = df1['multi_class'].fillna(0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [79]:
df1.head()

,time,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,rpm_mean,Asset,multi_class,Binary,stage,Unnamed: 0,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,s_temp_crank*s_pressure_crank,d_temp_crank*d_pressure_crank
0,01-12-2019 00:30,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238676,820.900548,1357.93250,Ashland_Unit256,Normal,0.0,3.0,NaN,35495.693668,178966.511738,36568.496401,181230.701112
1,01-12-2019 01:30,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,1358.24000,Ashland_Unit256,Normal,0.0,3.0,NaN,34287.262879,176957.908138,35230.156162,179385.590107
2,01-12-2019 02:30,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,1398.28125,Ashland_Unit256,Normal,0.0,3.0,NaN,33953.168175,177511.639298,34687.769610,179829.918423
3,01-12-2019 03:30,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,1397.87125,Ashland_Unit256,Normal,0.0,3.0,NaN,36123.580608,185232.534888,36900.189126,187543.369569
4,01-12-2019 04:30,1.0,109.567843,227.339240,351.392952,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,1397.31000,Ashland_Unit256,Normal,0.0,3.0,NaN,37697.983717,191259.228667,38501.367781,193516.619913


In [80]:
df1.shape

(102522, 21)

In [81]:
df1.columns

Index(['time', 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',
       's_pressure_mean', 'd_pressure_mean', 'rpm_mean', 'Asset',
       'multi_class', 'Binary', 'stage', 'Unnamed: 0',
       's_temp_mean*s_pressure_mean', 'd_temp_mean*d_pressure_mean',
       's_temp_crank*s_pressure_crank', 'd_temp_crank*d_pressure_crank'],
      dtype='object')

In [82]:
X_columns = [ 'cyl', 's_temp_crank', 'd_temp_crank', 's_pressure_crank',\
       'd_pressure_crank', 'rpm_crank', 's_temp_mean', 'd_temp_mean',\
       's_pressure_mean', 'd_pressure_mean','s_temp_mean*s_pressure_mean','d_temp_mean*d_pressure_mean', 'rpm_mean']

In [83]:
len(X_columns)

13

In [84]:
X = df1[X_columns]

In [85]:
X.shape

(102522, 13)

# build  with mean

In [86]:
X

,cyl,s_temp_crank,d_temp_crank,s_pressure_crank,d_pressure_crank,rpm_crank,s_temp_mean,d_temp_mean,s_pressure_mean,d_pressure_mean,s_temp_mean*s_pressure_mean,d_temp_mean*d_pressure_mean,rpm_mean
0,1.0,103.113613,216.591245,354.642761,836.740658,1357.93250,103.113613,218.012416,344.238676,820.900548,35495.693668,178966.511738,1357.932500
1,1.0,99.965045,214.136948,352.424753,837.714329,1358.24000,99.965045,215.236945,342.992523,822.153967,34287.262879,176957.908138,1358.240000
2,1.0,98.503206,213.432090,352.148637,842.562702,1398.28125,98.503206,214.106646,344.690998,829.080473,33953.168175,177511.639298,1398.281250
3,1.0,104.620153,220.973225,352.706318,848.715357,1397.87125,104.620153,221.697270,345.283193,835.520144,36123.580608,185232.534888,1397.871250
4,1.0,109.567843,227.339240,351.392952,851.224013,1397.31000,109.567843,228.114859,344.060655,838.433890,37697.983717,191259.228667,1397.310000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104454,2.0,57.404187,192.870092,45.099492,173.448757,1097.04375,57.404187,194.540939,45.360622,176.846614,2603.889623,34403.906401,1095.845625
104455,2.0,58.232545,192.783737,44.219374,169.162607,1094.24000,58.232545,195.095136,44.179864,172.503697,2572.705921,33654.632254,1094.654375
104456,2.0,58.345895,193.720827,43.146701,166.990013,1096.65000,58.345895,195.398975,43.390119,170.251174,2531.635307,33266.904953,1096.146250
104457,2.0,57.941348,194.445306,42.829713,167.715724,1095.04250,57.941348,196.355408,42.965954,170.994143,2489.505311,33575.624776,1093.078125


In [87]:
y_true = df1.iloc[:,14]

In [88]:
y_true.unique()

array([0., 1.])

In [89]:
#seed = 42
test_size = 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=test_size,shuffle= True)

In [90]:
# X_val, X_test, y_val, y_test = train_test_split(X_Test, y_Test, test_size=0.75)

In [91]:
from collections import Counter

In [92]:
print(X_train.shape,y_train.shape)

(87143, 13) (87143,)


In [93]:
len(X_test)

15379

In [94]:
#len(X_val)

In [95]:
Counter(y_train)

Counter({0.0: 56272, 1.0: 30871})

64->16->4->1

In [96]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 15.
        self.layer_1 = nn.Linear(13, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_3 = nn.Linear(64, 64)
        self.layer_4 = nn.Linear(64, 10)
        self.layer_5 = nn.Linear(10, 4)
        self.layer_out = nn.Linear(4, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.batchnorm4 = nn.BatchNorm1d(10)

        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.relu(self.layer_4(x))
        x = self.batchnorm4(x)
        x = self.relu(self.layer_5(x))
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [97]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
#X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [98]:
X_test

array([[ 1.86655328,  1.58263309,  2.45005154, ..., -1.16566168,
        -0.59397222,  0.55308   ],
       [-0.19972154,  0.1497499 , -0.6381731 , ...,  0.61055504,
        -0.23582031,  0.11775558],
       [ 0.83341587,  0.38963822,  0.53832168, ..., -0.73256293,
        -0.40842423,  0.05454887],
       ...,
       [ 0.83341587, -0.21206788, -1.22831581, ..., -1.31243389,
        -1.29724072,  0.59993335],
       [-1.23285894, -0.60247581, -0.30942601, ..., -0.18370012,
         0.01047001,  0.12453994],
       [-1.23285894,  0.11897336, -0.82394936, ...,  0.63139096,
        -0.38092275,  0.06735613]])

In [99]:
EPOCHS = 100
BATCH_SIZE = 75
LEARNING_RATE = 0.0792

In [100]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

y_train = y_train.values
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
# ## val data    
# class valData(Dataset):
    
#     def __init__(self, X_data):
#         self.X_data = X_data
        
#     def __getitem__(self, index):
#         return self.X_data[index]
        
#     def __len__ (self):
#         return len(self.X_data)

test_data = testData(torch.FloatTensor(X_test))
# val_data = valData(torch.FloatTensor(X_val))

In [101]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(dataset=val_data, batch_size=1)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [102]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [103]:
model = binaryClassification()
model.to(device)

binaryClassification(
  (layer_1): Linear(in_features=13, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [104]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [105]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [106]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        #X_batch = X_batch
  
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

In [107]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [108]:
confusion_matrix(y_test, y_pred_list)

array([[9269,  715],
       [ 704, 4691]])

# Old Assets

In [109]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93      9984
         1.0       0.87      0.87      0.87      5395

    accuracy                           0.91     15379
   macro avg       0.90      0.90      0.90     15379
weighted avg       0.91      0.91      0.91     15379



In [110]:
import pickle
filename = 'binarynn_dcp_model.pkl'

In [111]:
torch.save(model.state_dict(),filename)


In [112]:
model.load_state_dict(torch.load(filename))

<All keys matched successfully>

In [113]:
model.eval()

binaryClassification(
  (layer_1): Linear(in_features=13, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=64, bias=True)
  (layer_4): Linear(in_features=64, out_features=10, bias=True)
  (layer_5): Linear(in_features=10, out_features=4, bias=True)
  (layer_out): Linear(in_features=4, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

# Test Pred New Assets

In [114]:
X = df2[X_columns]

In [115]:
y_true = df2.iloc[:,14]

In [116]:
X_test = scaler.fit_transform(X)

In [117]:
test_data = testData(torch.FloatTensor(X_test))

In [118]:
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [119]:
X_test.shape

(5223, 13)

In [120]:
y_pred_list = []
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())



# New Assets

In [121]:
predictions=[]
predictions.append([y_pred_list[i][0][0] for i in range(len(y_pred_list))])

In [122]:
print(classification_report(y_true, predictions[0]))

              precision    recall  f1-score   support

         0.0       0.29      0.31      0.30      1937
         1.0       0.58      0.55      0.57      3286

    accuracy                           0.46      5223
   macro avg       0.43      0.43      0.43      5223
weighted avg       0.47      0.46      0.47      5223

